# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

488

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

### Extraction des trigrams

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 33min 37s
Wall time: 41min 13s


### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [32]:
model.wv["belgique"]

array([ 1.7810311 ,  0.3532156 ,  0.37565678, -1.3805081 , -7.6521244 ,
       -5.588041  ,  0.46658254, -3.9814672 , -1.2894727 ,  2.4892647 ,
       -2.489503  , -0.51785856, -0.4197926 ,  7.1481323 , -3.040979  ,
        4.8498654 , -1.4016389 ,  1.2591149 ,  4.7002993 , -4.7255616 ,
        2.4298477 , -1.1489832 , -0.38148236, -0.6713553 , -1.1096958 ,
       -0.06486624,  1.5873924 ,  4.013479  , -2.7588248 , -2.1892138 ,
       -0.98624843, -3.883302  ], dtype=float32)

### Calculer la similarité entre deux termes

In [25]:
model.wv.similarity("belgique", "france")

0.9301486

In [26]:
model.wv.similarity("guerre", "paix")

0.62399405

In [28]:
model.wv.similarity("homme", "femme")

0.6203839

In [27]:
model.wv.similarity("mort", "suicide")

0.6274558

In [33]:
model.wv.similarity("football", "tennis")

0.7392186

### Chercher les mots les plus proches d'un terme donné

In [30]:
model.wv.most_similar("belgique", topn=10)

[('france', 0.9301486611366272),
 ('hollande', 0.8700990676879883),
 ('belgiquo', 0.8545124530792236),
 ('tchecoslovaquie', 0.8395277857780457),
 ('suisse', 0.8327073454856873),
 ('grece', 0.8264899253845215),
 ('yougoslavie', 0.8225748538970947),
 ('tunisie', 0.8217123746871948),
 ('sarre', 0.8050734400749207),
 ('belglquo', 0.802809476852417)]

In [31]:
model.wv.most_similar("guerre", topn=10)

[('liberation', 0.9023601412773132),
 ('guerro', 0.8798238635063171),
 ('derniere_guerre', 0.8769397735595703),
 ('guorre', 0.8611417412757874),
 ('population', 0.853924036026001),
 ('guerre_mondiale', 0.829509973526001),
 ('colonie', 0.8134241700172424),
 ('wehrmacht', 0.8110015392303467),
 ('capitulation', 0.8099997043609619),
 ('reddition', 0.804178774356842)]

In [38]:
model.wv.most_similar("science")

[('psychologie', 0.9309033155441284),
 ('democratie', 0.9084911346435547),
 ('doctrine', 0.9026889204978943),
 ('tolerance', 0.901496410369873),
 ('civilisation', 0.8962649703025818),
 ('fidelite', 0.8944718241691589),
 ('comprehension', 0.8931295871734619),
 ('litterature', 0.8894110321998596),
 ('decadence', 0.8864009380340576),
 ('noblesse', 0.8828279972076416)]

### Faire des recherches complexes à travers l'espace vectoriel

In [40]:
print(model.wv.most_similar(positive=['maroc', 'tunisie'], negative=['europe']))

[('danemark', 0.8252747058868408), ('venezuela', 0.8251700401306152), ('liban', 0.8169543743133545), ('ghana', 0.8122550845146179), ('portugal', 0.8068138360977173), ('pakistan', 0.7912518978118896), ('tonkin', 0.7902986407279968), ('mexique', 0.7832223773002625), ('guatemala', 0.7803775072097778), ('bolivie', 0.778136670589447)]


In [46]:
print(model.wv.most_similar(positive=['avion', 'bateau']))

[('navire', 0.9100179076194763), ('hydravion', 0.874131977558136), ('paquebot', 0.8715959787368774), ('aeroplane', 0.8674670457839966), ('convoi', 0.8637217283248901), ('helicoptere', 0.8579516410827637), ('canot', 0.8518037796020508), ('bord', 0.84962397813797), ('yacht', 0.8488996624946594), ('steamer', 0.8447496891021729)]


In [47]:
print(model.wv.most_similar(positive=['moderne', 'technologie']))

[('utilitaire', 0.8611147999763489), ('heraldique', 0.8601410388946533), ('decoration', 0.8209235072135925), ('ventilation', 0.8088339567184448), ('didactique', 0.8016582727432251), ('artistique', 0.7844574451446533), ('sculpture', 0.7804416418075562), ('chimie', 0.7798225283622742), ('harmonique', 0.7723403573036194), ('architectural', 0.7715532183647156)]
